In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [63]:
def get_grant_features(inn):
    query = f'https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/cards?SearchString={inn}&Statuses%5B0%5D.Selected=true&Statuses%5B0%5D.Name=%D0%BF%D0%BE%D0%B1%D0%B5%D0%B4%D0%B8%D1%82%D0%B5%D0%BB%D1%8C+%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D1%81%D0%B0&Statuses%5B1%5D.Name=%D0%BD%D0%B0+%D0%BD%D0%B5%D0%B7%D0%B0%D0%B2%D0%B8%D1%81%D0%B8%D0%BC%D0%BE%D0%B9+%D1%8D%D0%BA%D1%81%D0%BF%D0%B5%D1%80%D1%82%D0%B8%D0%B7%D0%B5&Statuses%5B2%5D.Name=%D0%BF%D1%80%D0%BE%D0%B5%D0%BA%D1%82+%D0%BD%D0%B5+%D0%BF%D0%BE%D0%BB%D1%83%D1%87%D0%B8%D0%BB+%D0%BF%D0%BE%D0%B4%D0%B4%D0%B5%D1%80%D0%B6%D0%BA%D1%83&RegionId=&ContestDirectionTenantId=&IsNormalTermProjects=true&IsLongTermProjects=true&CompetitionId=&DateFrom=&DateTo=&Statuses%5B0%5D.Selected=false&Statuses%5B1%5D.Selected=false&Statuses%5B2%5D.Selected=false&IsNormalTermProjects=false&IsLongTermProjects=false'
    main_request = requests.get(query, headers = {'User-Agent': 'Googlebot'})
    main_page = BeautifulSoup(main_request.text)
    
    all_features = {}

    for link in main_page.findAll('a'):
            features = {}
            if 'href' in link.attrs:
                if 'public/application/item?' in link['href']:
                    grant_link = 'https://xn--80afcdbalict6afooklqi5o.xn--p1ai' + link['href']
                    request = requests.get(grant_link, headers = {'User-Agent': 'Googlebot'})   
                    page = BeautifulSoup(request.text)

                    for li in page.findAll('li', class_='winner-info__list-item'):
                        feature_name = li.find('span', class_='winner-info__list-item-title').text
                        feature_value = li.find('span', class_='winner-info__list-item-text').text
                        feature_value = re.sub(r'\s+', ' ', feature_value).strip()
                        features[feature_name] = feature_value

                    for li in page.findAll('li', class_='circle-bar__info-item'):
                        feature_name = li.find('span', class_='circle-bar__info-item-title').text
                        feature_value = li.find('span', class_='circle-bar__info-item-number').text
                        feature_value = re.sub(r'\s+', ' ', feature_value).strip()
                        features[feature_name] = feature_value

                    text_description = page.findAll('div', class_='winner__details-box')

                    for block in text_description:    
                        feature_name = block.h2.text.strip()
                        feature_value = re.sub('\s+', ' ', block.text).replace(feature_name, '').strip()
                        features[feature_name] = feature_value
            if features:        
                all_features[grant_link] = features
                
    feat_df = pd.DataFrame.from_dict(all_features, orient='index')
    return feat_df

In [64]:
def normalize_decimal_cols(df):
    float_cols = ['Размер гранта', 
                  'Общая сумма расходов на реализацию проекта', 
                  'Cофинансирование', 
                  'Перечислено Фондом на реализацию проекта', 
                  'Рейтинг заявки']
    
    for col in float_cols:
        if col in list(df):
            if col == 'Рейтинг заявки':
                df[col] = df[col].apply(lambda x: float(x.replace(',', '.')) if x else x)
            else:
                df[col] = df[col].apply(lambda x: float(''.join(re.findall('[0-9]+', x.split(',')[0]))) if x else x)
    return df

In [67]:
def grant_info_flg(inn):
    grant_info = get_grant_features(inn)
    if len(grant_info) > 0:
        return True
    else:
        return False
    
    if isinstance(grant_info, pd.DataFrame):
        return True
    else:
        return False

In [68]:
grant_info_flg(inn)

True

In [92]:
inn = '2222848892'

df = get_grant_features(inn)
df = normalize_decimal_cols(df)
df.reset_index(inplace=True)

/opt/conda/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


In [96]:
list(df[['index', 'Рейтинг заявки', 'Размер гранта', 'Cофинансирование', 'Цель']].values)

[array(['https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/item?id=E6CA268D-1EBE-424D-A88F-D6F85FB6E9CB',
        71.17, 5764085.0, 7610433.0,
        'Организация регулярной комплексной поддержки и сопровождения в преодолении кризисной ситуации для малообеспеченных и многодетных семей на базе районных "узловых центров" не менее чем в 10 населенных пунктах Алтайского края и Республики Алтай в период февраль 2021-январь 2022 гг.'],
       dtype=object),
 array(['https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/item?id=EE703719-C598-4317-BFEA-A1449850A778',
        72.0, 2927552.0, 3057827.0,
        'Создание условий для непрерывного посещения спортивных секций, укрепления здоровья и повышения уровня жизни не менее чем для 150 детей в возрасте от 4 до 15 лет из малообеспеченных и многодетных семей, проживающих на территории Алтайского края в 2020-2021 гг.'],
       dtype=object),
 array(['https://xn--80afcdbalict6afooklqi5o.xn--p1ai/public/application/item?i

In [88]:

df

,level_0,index,Конкурс,Грантовое направление,Рейтинг заявки,Номер заявки,Дата подачи,Сроки реализации,Организация,ИНН,...,Размер гранта,Cофинансирование,Перечислено Фондом на реализацию проекта,Краткое описание,Цель,Задачи,Обоснование социальной значимости,География проекта,Целевые группы,Контактная информация
0,0,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2021,"Социальное обслуживание, социальная поддержка ...",71.17,21-1-001577,12.10.2020,01.02.2021 - 31.01.2022,"ФОНД ПОМОЩИ МНОГОДЕТНЫМ И МАЛОИМУЩИМ СЕМЬЯМ, Д...",2222848892,...,5764085.0,7610433.0,5764085.0,Сердце Алтая - проект социальной комплексной п...,Организация регулярной комплексной поддержки и...,"Организация работы районных ""узловых центров"" ...","Зарубежные исследования показывают, что дети, ...","Алтайский край (г. Барнаул + г. Новоалтайск, г...",Малообеспеченные и многодетные семьи Алтайског...,"{""address"":""656043, г Барнаул, ул Анатолия, д ..."
1,1,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Второй конкурс 2020,"Охрана здоровья граждан, пропаганда здорового ...",72.00,20-2-000118,04.04.2020,01.07.2020 - 30.06.2021,"ФОНД ПОМОЩИ МНОГОДЕТНЫМ И МАЛОИМУЩИМ СЕМЬЯМ, Д...",2222848892,...,2927552.0,3057827.0,2927552.0,"Проект ""Поддержи Чемпиона"" направлен на предос...",Создание условий для непрерывного посещения сп...,Разработка нормативных и учебно-методических м...,"В докладе ""О роли субъектов Российской Федерац...","Алтайский край (г. Барнаул, г. Бийск, г. Зарин...",дети в возрасте от 4 до 15 лет из малообеспече...,"{""address"":""656043, г Барнаул, Социалистически..."
2,2,https://xn--80afcdbalict6afooklqi5o.xn--p1ai/p...,Первый конкурс 2020,"Социальное обслуживание, социальная поддержка ...",65.25,20-1-002398,20.11.2019,01.03.2020 - 31.12.2020,"ФОНД ПОМОЩИ МНОГОДЕТНЫМ И МАЛОИМУЩИМ СЕМЬЯМ, Д...",2222848892,...,2232214.0,917220.0,2232214.0,"""Школа жизни"" - программа Фонда ""Облака"", приз...","Проведение образовательной программы ""Школа Жи...",Разработка пакета нормативных и учебно-методич...,Численность россиян с денежными доходами ниже ...,Алтайский край,малоимущие и многодетные женщины с маленькими ...,"{""address"":""656043, г Барнаул, Социалистически..."
